## Tracking the International Space Station with Dask

In this notebook, we will use two APIs: [Google Maps Geocoder](https://developers.google.com/maps/documentation/geocoding/) and the [open notify API for ISS location](http://api.open-notify.org/). We will use them to track the ISS location and next pass time in relation to a list of cities.

To help build our graphs and intelligently parallelize data, we will use [Dask](http://dask.pydata.org/en/latest/), specifically [Dask delayed](http://dask.pydata.org/en/latest/delayed.html).

In [ ]:
import requests
import logging
import sys
import numpy as np
from time import sleep
from datetime import datetime
from math import radians
from dask import delayed
from operator import itemgetter
from sklearn.neighbors import DistanceMetric

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### First, we need to get lat and long pairs from a list of cities

In [ ]:
def get_lat_long(address):
    resp = requests.get(
        'https://eu1.locationiq.org/v1/search.php',
        params={'key': '92e7ba84cf3465', #Please be kind, you can generate your own for more use here - https://locationiq.org :D
                'q': address,
                'format': 'json'}
    )
    if resp.status_code != 200:
        print('There was a problem with your request!')
        print(resp.content)
        return
    data = resp.json()[0]
    return {
        'name': data.get('display_name'),
        'lat': float(data.get('lat')),
        'long': float(data.get('lon')),
    }

In [ ]:
get_lat_long('Berlin, Germany')

In [ ]:
locations = []
for city in ['Seattle, Washington', 'Miami, Florida', 
             'Berlin, Germany', 'Singapore', 
             'Wellington, New Zealand',
             'Beirut, Lebanon', 'Beijing, China', 'Nairobi, Kenya',
             'Cape Town, South Africa', 'Buenos Aires, Argentina']:
    locations.append(get_lat_long(city))
    sleep(2)

In [ ]:
locations

### Now we can define the functions we will use to get the ISS data and compare location and next pass times amongst cities 

In [ ]:
def get_spaceship_location():
    resp = requests.get('http://api.open-notify.org/iss-now.json')
    location = resp.json()['iss_position']
    return {'lat': float(location.get('latitude')),
            'long': float(location.get('longitude'))}

In [ ]:
def great_circle_dist(lon1, lat1, lon2, lat2):
    "Found on SO: http://stackoverflow.com/a/41858332/380442"
    dist = DistanceMetric.get_metric('haversine')
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    X = [[lat1, lon1], [lat2, lon2]]
    kms = 6367
    return (kms * dist.pairwise(X)).max()

In [ ]:
def iss_dist_from_loc(issloc, loc):
    distance = great_circle_dist(issloc.get('long'), 
                                 issloc.get('lat'), 
                                 loc.get('long'), loc.get('lat'))
    logging.info('ISS is ~%dkm from %s', int(distance), loc.get('name'))
    return distance

In [ ]:
def iss_pass_near_loc(loc):
    resp = requests.get('http://api.open-notify.org/iss-pass.json',
                        params={'lat': loc.get('lat'), 
                                'lon': loc.get('long')})
    data = resp.json().get('response')[0]
    td = datetime.fromtimestamp(data.get('risetime')) - datetime.now()
    m, s = divmod(int(td.total_seconds()), 60)
    h, m = divmod(m, 60)
    logging.info('ISS will pass near %s in %02d:%02d:%02d',loc.get('name'), h, m, s)
    return td.total_seconds()

In [ ]:
iss_dist_from_loc(get_spaceship_location(), locations[4])

In [ ]:
iss_pass_near_loc(locations[4])

### Let's create a delayed pipeline

In [ ]:
output = []

for loc in locations:
    issloc = delayed(get_spaceship_location)()
    dist = delayed(iss_dist_from_loc)(issloc, loc)
    output.append((loc.get('name'), dist))

closest = delayed(lambda x: sorted(x, 
                                   key=itemgetter(1))[0])(output)

In [ ]:
closest

### Let's see our DAG!

In [ ]:
closest.visualize()

### Remember: it is lazy, so let's start it with `compute()`

In [ ]:
closest.compute()

### Exercise: which city will it fly over next?

### Extra: add your city and compare!

In [ ]:
# %load ../solutions/dask.py

